In [ ]:
import os
from urllib.parse import urlparse, parse_qs
from fpdf import FPDF
from IPython.display import FileLink
from datetime import datetime
from arcgis.gis import GIS
from arcgis.features import FeatureLayer

# Function to add content to PDF
def add_content_to_pdf(pdf, content):
    for line in content.split("\n"):
        pdf.cell(0, 10, line, ln=True)

# Function to generate PDF
def generate_pdf(globalid):
    # Get credentials from environment variables
    username = os.getenv('AGOL_USERNAME')
    password = os.getenv('AGOL_PASSWORD')

    # Connect to ArcGIS Online
    gis = GIS("https://www.arcgis.com", username, password)

    # Get the feature layer
    feature_layer_url = "https://services1.arcgis.com/fbjmreFs181B7jr9/arcgis/rest/services/Recent_Solar_Panel_Row_Inspections/FeatureServer/0"
    feature_layer = FeatureLayer(feature_layer_url)

    try:
        # Query the feature by globalid
        features = feature_layer.query(where=f"GlobalID='{globalid}'", out_fields="*").features
    except Exception as e:
        return f"Error querying the feature layer: {str(e)}"

    if not features:
        return "Feature not found"

    feature = features[0]
    attributes = feature.attributes

    # Initialize PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Convert timestamp to readable date format
    inspection_date = attributes.get('InspectionDate', '')
    if inspection_date:
        inspection_date = datetime.fromtimestamp(inspection_date / 1000).strftime('%Y-%m-%d %H:%M:%S')

    # Simulate the Arcade expression
    content = "Solar CM Inspection Log\n\n"
    content += f"Inspector: {attributes.get('Inspector', '')}\n"
    content += f"Inspection Date / Time: {inspection_date}\n\n"

    def section_header(title):
        return title + "\n\n"

    def check_fields(fields):
        for field in fields:
            value = attributes.get(field)
            if value not in [None, '', ' ']:
                return True
        return False

    def add_fields_to_content(fields, labels):
        field_content = ""
        for field, label in zip(fields, labels):
            value = attributes.get(field)
            if value:
                field_content += f"{label}: {value}\n"
        return field_content

    sections = [
        ("Safety", ["HardHat", "HighVis", "EyeFaceZ87", "FallPro", "HandPro", "FootPro", "HearPro"],
         ["Hard Hat", "High Visibility Vests", "Eye / Face Protection Z-87+", "Fall Protection / Harness & Retrac",
          "Hand Protection (cut resistant)", "Foot Protection (Steel toe)", "Hearing Protection"]),

        ("Housekeeping", ["WorkArea"], ["Work Area Clean"]),

        ("Hole-Fire Watch or Spotters", ["Spotter", "HoleFireWatch"], ["Spotter", "Hole & Fire Watch"]),

        ("Permits", ["WorkPerm"], ["Work Permit On Site"]),

        ("Air Tools Inspection", ["HoseConnPin", "GrindChipgun"], ["Hoses, connections & pins", "Grinders & chipping guns"]),

        ("Electrical Tools", ["ElecCordConn", "GrndFaultCirInter"],
         ["Electric cords & Connectors", "Ground Fault Circuit Interrupter"]),

        ("Hand Tools", ["HandTools", "HandToolsUse"], ["Hand tools in Good Condition", "Proper Use (right tool – right job)"]),

        ("Welding & Cutting", ["CutRigs", "WeldMach", "CutHoseWeldLead"],
         ["Cutting Rigs (condition) / Storage", "Welding Machines (condition)", "Cutting Hoses & Welding Leads"]),

        ("Safety Equipment", ["FireEx", "ResPPEDust", "ChemPro", "ChemKit"],
         ["Fire Extinguishers at Hand", "Respiratory PPE & Dust Control", "Chemical Protection", "Spill Response Kit (on site)"]),

        ("Work Surfaces", ["Scaffolds"], ["Scaffolds (construction) / Condition"]),

        ("Vehicles and Equipment", ["VehCon", "EquipInspect"], ["Vehicle Condition", "Equipment Inspection"]),

        ("Barricades", ["BarrUseRem"], ["Use & Removal"]),

        ("Materials", ["MSDS", "MatStor", "MatHand"], ["MSDS On Site", "Material Storage", "Material Handling"]),

        ("Safe Work Practices", ["EmAccess", "Confined", "HotWork", "SewerCov", "DrinkCon"],
         ["Emergency Access to All Areas", "Working in Confined Spaces", "Hot Work", "Are Sewers Covered?", "Drinking Water Containers"]),

        ("Employee Training", ["TrainCard", "ExplainPlan", "SDSwhere"],
         ["Training Card On-Person", "Explain the Unit Evacuation Plan", "Where would you get a SDS"]),

        ("Environment", ["PrevSpillLeak", "PropWaste"], ["Prevention of Spills & Leaks", "Proper Handling of Waste"]),

        ("Site Conditions", ["WeatherCon", "NumEmp"], ["Weather Conditions", "Number of Employees On Site"]),

        ("Daily Report", ["Comments"], ["Comments"]),

        ("Quality", ["Location", "ConstStage", "InInspRain", "SWPPPNotice", "BestManage", "BOPEPCDoc"],
         ["Location", "Stage of Construction", "In inspection routine or due to rain event?", "SWPPP Notice is poster near site entrance?",
          "Are the Best Management Practices in place per the approved SWPPP?", "Is BOP-EPC SWPPP documentation stored on site and up to date?"]),

        ("Site Cut/Fill Inspection", ["PermitsExGrade", "BenchExGrade", "EarthworkExGrade", "UnderExGrade", "StructExGrade", "DrainExGrade",
                                      "BackwaterExGrade", "SoilProExGrade", "DewaterExGrade", "EarthExGrade", "LiftCompactBackfill", "CompactBackfill"],
         ["All required permits have been obtained", "Existing benchmarks and baseline is not tampered with",
          "Boundaries and limits of earthwork are established and verified against design drawings",
          "All underground utilities and structures have been identified and marked",
          "Arrangements have been made to protect the existing utilities, trees and structures in and around the excavation",
          "Excavation and construction area does not impede natural drainage",
          "Excavation and construction area does no backwater on to adjoining properties", "Adequate soil protection is implemented",
          "Excavation free of excessive moisture and adequate dewatering mechanism is installed", "Excavated earth is stockpiled at designated location",
          "Lifts and compaction of backfill shall be carried out in accordance with project specification",
          "The specified backfill material is compacted with specified density, moisture tolerance and specified lift thickness"]),

        ("New Road Construction", ["RoadSectBegin", "RoadSectEnd", "VerifyNatSoil", "SoilCondChg", "ProctorMat", "VerifySfcVeg1",
                                   "VerifyGeoTex", "VerifySubgrade", "VerifyProofRoll", "VerifySubgrade1", "VerifyAggregate", "VerifyAggregate2",
                                   "RoadWDSpec", "RoadSfc"],
         ["Road Section designation beginning", "Road Section designation end", "Verify proctors have been taken of native soil with the road alignment",
          "Has another proctor been requested if soil conditions have changed?", "Has proctor been taken from imported materials?",
          "Verify surface vegetation is removed to the depth required by drawing or specification below the ground surface",
          "Verify Geotextile material is installed if required by drawing or specification",
          "Verify roadway roadbed (subgrade) is compacted to maximum dry density and optimum water content",
          "Verify proof roll meets the requirements of drawing or specification and test has been observed by QC or third party testing agency",
          "Verify roadway roadbed (subgrade) is installed in required lifts thickness and compacted, and number of tests is adequate per specification and drawings",
          "Verify compacted rock fill / roadway aggregate base course is installed in required lifts thickness prior to compaction",
          "Verify compacted rock fill / roadway aggregate base course is compacted and meets relative density or proof roll standards",
          "Is road width and depth to design specifications?", "Is the finished road surface acceptable per design?"]),

        ("Culverts", ["CulvMatSz", "CulvLoc", "DrainCulvertMatl"],
         ["Verify culvert material and size comply with project specifications", "Verify culvert locations are as per approved drawings and as-builts",
          "Verify that top of culvert material complies with road specification (when applicable)"]),

        ("Fence Installation", ["VerifyFenceGates", "VerifyGateOpen", "VerifyLinePosts", "VerifyLPAnchor", "VerifyTermGate", "SecSide",
                                "VerifyFabTies", "VerifySecGate", "VerifyTensionWires", "Concrete"],
         ["Verify fence gates, corners, and posts are in correct location as shown in drawings",
          "Verify gate openings are installed in accordance with drawing and are at the locations shown on the plan drawings",
          "Verify line posts are spaced center to center, according to drawings", "Verify line post anchorage is according to drawing and filled with concrete if required",
          "Verify terminal gate, end and corner posts anchorage holes are according to drawing, and filled with concrete if required",
          "Wire ties, rails, post and braces are constructed on the secure side of the fence (inside of plant) and chain link fabric is on non-secure side (outside of plant)",
          "Verify fabric ties are installed on each post and tension wires are installed according to drawing",
          "Verify security swing gate, isolation joints and ditch crossings are installed according to drawing",
          "Verify tension wires are installed before the fabric and wire is installed at the top of the fabric and approximately 6 inches above grade",
          "Concrete mix matches specifications"]),

        ("Material Offload – Modules", ["ShipCont", "PalletDistribute", "ContractInspect", "DelQuant", "DamagedModules"],
         ["Shipping container arrived without any visible damage?",
          "Contractor is offloading and staging pallets correctly per the benchmarked process?",
          "Contractors is inspection pallets for damage, confirming quantities are in line with the BOL?",
          "Delivered quantities are tracked by Contractor and BOL is added to delivery log",
          "Damaged or missing modules are tracked by Contractor and reported to Owner immediately?"]),

        ("Module Installation", ["ModClearance", "ToolCalibrate", "Procedures", "ModDamageFree", "ModSecured"],
         ["Are proper clearances being maintained?", "Are all applicable tools being used calibrated?",
          "Were proper procedures followed during installation to avoid damage?", "Are modules free of damage and/or defects?",
          "Are modules properly secured, verified, and marked?"]),

        ("Racking / Trackers", ["TrackClearance", "ToolCal", "hardware", "HardwareTorq"],
         ["Are the correct clearances being maintained?", "Do all tools being used have current calibration certificates?",
          "Is the correct hardware being used for installation?", "Is all hardware torqued to correct specifications, verified and marked?"]),

        ("DC CAB (above ground) Inspection", ["StructEndPost", "StructEndPost2", "StructMess", "StructMessWire", "StructCAB1", "StructCAB2", "StructNoDam",
                                              "ElectOEM", "ElectCond", "ElectStrain", "ElectCondSeat", "ElectCondPro", "ElectCondLabel", "ElectCondDam",
                                              "ElectCondOver", "ElectSysBond", "ElectConnVer", "ElectServEquip", "ElectServEquipSeal"],
         ["End posts match drawings and specifications", "End posts are installed per drawings",
          "Messenger wire fasteners are installed per drawings", "Messenger wire utilized matches drawings and specifications",
          "CAB hangers match drawings and specifications", "CAB hangers are spaced per drawings", "Verify no physical damage to installation or components",
          "All equipment has been handled and staged per OEM guidelines, manuals, etc.", "Conductors match drawings and specifications",
          "Strain relief provided as required", "Conductors seated in appropriate hanger section", "Conductors protected from physical damage",
          "Conductors labeled per drawings", "Verify no physical damage to conductors", "Verify no physical damage to overmolds",
          "System is bonded and grounded per plan", "All connectors verified for proper support, seating, strain relief and protection from damage",
          "Service equipment labeled per drawings", "Service equipment sealed from ingress of dirt, debris and rodents"]),

        ("Cable Inspection", ["CabBund", "CabDamage", "CabCondDamage", "CabCondSize", "CabBedMatl", "CabMinDepth", "CabMinClear",
                              "CabFibSpace", "CabMarkDepth", "CabBackfillMatl", "CabBackfillSpec"],
         ["Is the collection cable bundled in trefoil configuration?", "Is the collection cable undamaged?",
          "Was the conductor installed without damage?", "Is the conductor the correct size?", "Is the proper bedding material being used?",
          "Was the minimum depth achieved?", "Do all the cables have minimum required clearances?", "Was the interduct/fiber spaced correctly?",
          "Was the marker installed at the correct depth?", "Is proper backfill material being used?", "Did the compaction and backfill effort meet applicable specifications?"]),

        ("Collection Boring Inspection", ["CabBund2", "CabColDamage", "CabCondDamage2", "CabCondSize2", "CabMinDepth2", "CabBellComplete",
                                          "CabPullLube", "CabBoreMin"],
         ["Is the collection cable bundled in trefoil configuration?", "Is the collection cable undamaged?",
          "Was the conductor installed without damage?", "Is the conductor the correct size?", "Is the minimum depth at entrance and exit achieved?",
          "Was bell end or router completed on bore conduit?", "Was pulling lubricant used for cable installation?", "Was the bore minimum width gained?"]),

        ("Junction Box", ["JBLevel", "JBCondSeal", "JBVDefects", "JBBollard", "JBMVTorq", "JBBoltOps", "JBFaultIndicators", "JBGroundComplete",
                          "SBStick", "JBslack", "JBCabLab", "JBRock", "JBPhaseDir", "JBCabDamaged", "JBCondDamaged", "JBSoilCompact", "JBMinDepth",
                          "JBCabMark", "JBCrossBrace", "JBCompactBackfill", "JBLocCollect", "JBNamePlate"],
         ["Is the junction box level?", "Are all conduits sealed?", "Is the junction box and vault free of defects, holes, chips, etc.? ",
          "Are bollards installed?", "Are MV terminations torque marked?", "Are the door, lock, and pentagon head bolt operational?",
          "Are the fault indicators installed?", "Is the grounding complete?", "Are all stickers installed in the correct locations?",
          "Do the MV cables have slack for two terminations?", "Are the cable labels (to / from) installed?", "Is the rock installed correctly?",
          "Is the collection cable entering and exiting the junction box phased A-B-C from left to right?", "Is the collection cable undamaged?",
          "Were any conductors damaged during installation?", "Is minimum depth of soil compacted per plans and specifications?",
          "Is the minimum depth of rock installed inside the vault per the plans and specifications?", "Are the cables for each phase marked correctly?",
          "Is the cross brace installed per manufacturer’s instructions?", "Were the compaction and backfill requirements met?",
          "Is the location collected for the purpose of as build drawings?", "Are all labels, warning signs, and fault indicators nameplates installed?"]),

        ("DC Collection Combiner Box / Load Break Disconnect", ["DCBox", "DCBoxMount", "DCGround", "DCCond", "DCCabLab", "LVCabLand",
                                                                "DCFuses", "DCCombiner", "DCStripTerm", "DCCabBend", "DCLugTorq"],
         ["Is the combiner box clean and free of visible damage?", "Has the combiner box been mounted according to plans and specifications?",
          "Is the grounding installed according to the specifications?", "Are conduits entering the combiner box installed and tightened according to specifications?",
          "Are all cables coming into the combiner box connected and labeled according to the specifications?", "Are all LV cables landed and secured on a bus?",
          "Are the required fuses installed?", "Is the combiner box sealed according to the specifications?", "Verify the cables are properly stripped and terminated correctly?",
          "Confirm that cables have the proper bend radii:", "Verify the lugs are torqued and marked:"]),

        ("Met Sensors – POA Pyranometer & BOM Temp", ["METPyr", "METPOA", "METMount", "METTempSens", "METBOMMount", "METCabLand", "METCondSeal",
                                                      "METCabManage", "METPower"],
         ["Verify both front and back facing pyranometers are installed in their correct locations according to the IFC drawings and are not shaded",
          "Verify POA’s are in alignment with the tracker row", "Verify mounting hardware is properly tightened",
          "Verify BOM temperature sensors are installed in their correct locations according to the IFC drawings",
          "Verify BOM’s are mounted using correct adhesive applied properly per the equipment specifications",
          "Ensure all cables have been landed and labeled correctly", "Ensure all conduits are sealed",
          "Ensure cable management is clean and complete. Exposed cables have protection loom if available",
          "Is equipment powered on, calibrated and fully commissioned?"]),

        ("Module Wiring", ["ModWireLabel", "ModWireSecure", "ModWeatherSeal", "ModWireAbrasion", "ModWireClearance", "ModSplitLoom",
                           "ModConnectSecure", "ModTugTest"],
         ["Are all wires properly labeled / marked?", "Are all wires secured properly with weather resistant ties?",
          "Are all weather-head openings sealed according to the specifications?", "Are all wires free from possible abrasion?",
          "Do all wires have minimum clearance according to spec and proper slack to allow tracker to operate in full range of motion?",
          "Is split loom installed on wires that are exposed to direct sunlight as per plans?", "Are module connectors securely connected together?",
          "Has tug test been performed on connectors? (Connectors must not be under load during test)"]),

        ("Drainage Structure", ["DrainMatl", "DrainLoc", "DrainCulvertMatl", "DrainInOut", "DrainFlowPath", "DrainObstruct"],
         ["Verify drainage structure material and size comply with specifications", "Verify drainage structure locations are as per approved drawings",
          "Verify that top of culvert material complies with road specification", "Verify that inlet and outlets are graded per approved drawings",
          "Is the path of flow free of scour or erosion?", "Is the culvert free of flow restrictions?"]),

        ("BESS LV Terminations", ["BESSGround", "BESSCond", "BESSCab", "BESSLandBus", "BESSFuses", "BESSStripTerm", "BESSCableRadii", "BESSLugs",
                                  "BESSCondSeal"],
         ["Is the grounding installed according to the specifications?", "Are conduits installed and tightened according to specifications?",
          "Are all cables connected and labeled according to specifications?", "Are all LV cables landed and secured on a bus?",
          "Are the required fuses installed?", "Verify that cables are properly stripped and terminated correctly",
          "Confirm that cables have the proper bend radii", "Verify lugs are torqued and marked", "Verify conduits have been sealed"])
    ]

    for section in sections:
        section_name, fields, labels = section
        print(f"Processing section: {section_name}")  # Debugging
        if check_fields(fields):
            content += section_header(section_name)
            section_content = add_fields_to_content(fields, labels)
            content += section_content
            print(f"Section: {section_name}\n{section_content}\n")  # Debugging
            content += "\n"

    print(f"Final Content:\n{content}")  # Debugging

    # Add content to PDF
    add_content_to_pdf(pdf, content)

    # Save the PDF
    pdf_filename = f"inspection_report_{globalid}.pdf"
    pdf.output(pdf_filename)

    return pdf_filename

# Function to get the GlobalID from the URL parameters
def get_globalid_from_url():
    # Get the full URL of the current notebook
    notebook_url = os.getenv('BINDER_REQUEST_URL', 'http://localhost:8888/notebooks/notebook.ipynb?GlobalID=40c2d63d-94b3-4e6c-a365-6db4944de448')
    
    # Parse the URL and extract the query parameters
    parsed_url = urlparse(notebook_url)
    query_params = parse_qs(parsed_url.query)
    
    # Get the GlobalID from the query parameters
    globalid = query_params.get('GlobalID', [None])[0]
    
    return globalid

# Use the function to get the GlobalID
globalid = get_globalid_from_url()
print(f"GlobalID: {globalid}")

# Proceed with the rest of your code using the globalid
if globalid:
    pdf_filename = generate_pdf(globalid)
    display(FileLink(pdf_filename))
else:
    print("GlobalID parameter is missing in the URL")